<a href="https://colab.research.google.com/github/inigo-rojo/colab/blob/main/MeasuringStsBenchmarkSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.8 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=f98473b8af450ed8a3e8eb19fb505a7ce41703073f9cc7998a93a2f91bcbd739
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler, models, evaluation, losses

In [ ]:
import os

In [ ]:
#g = open("/content/drive/My Drive/TFG/Datasets/test.txt", "w")
src_sentences = ["The technology-laced Nasdaq Composite Index .IXIC inched down 1 point, or 0.11 percent, to 1,650."]
trg_sentences = ["El índice Nasdaq Composite Index.IXIC bajó 1 punto, o 0,11 por ciento, a 1.650."]
#dev_trans_acc = evaluation.TranslationEvaluator(src_sentences, trg_sentences, name=os.path.basename(dev_file),batch_size=inference_batch_size)
dev_trans_acc = evaluation.TranslationEvaluator(src_sentences, trg_sentences, name="/content/drive/My Drive/TFG/Datasets/test.txt",batch_size=64)



using stsb-mpnet-base-v2 as it's the model with the greater stsb score in the listed models

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-mpnet-base-v2')

New file structure: metadata + original sentences + translated sentences + cosine similarities with the following pair order: s1 s2 , s1 ts1 , s1 ts 2, s2 ts1, s2 ts2, ts1 ts2

Rating the M2M100_418M translation

In [ ]:
f = open("/content/drive/My Drive/TFG/Datasets/sts/translatedNewsDevM2M100_418M.csv", "r")
sentences=f.readlines()
ratedSentences= ""
for sentence in sentences:
  sentencesSts1 = []
  sentencesSts2 = []
  sentence = sentence.split("\t")
  sentence[-1] = sentence[-1].replace("\n","")
  metaData = sentence[0:5]
  sent = sentence[5:]
  sentencesSts1.extend([sent[0],sent[0],sent[0],sent[1],sent[1],sent[2]])
  sentencesSts2.extend([sent[1],sent[2],sent[3],sent[2],sent[3],sent[3]])
  embeddings1 = model.encode(sentencesSts1, convert_to_tensor=True)
  embeddings2 = model.encode(sentencesSts2, convert_to_tensor=True)
  cosine_scores = util.cos_sim(embeddings1, embeddings2)
  ratedSentences += metaData[0]+"\t"+metaData[1]+"\t"+metaData[2]+"\t"+metaData[3]+"\t"+metaData[4]+"\t"+sent[0]+"\t"+sent[1]+"\t"+sent[2]+"\t"+sent[3]+"\t"+"{:.4f}".format(cosine_scores[0][0])+"\t"+"{:.4f}".format(cosine_scores[1][1])+"\t"+"{:.4f}".format(cosine_scores[2][2])+"\t"+"{:.4f}".format(cosine_scores[3][3])+"\t"+"{:.4f}".format(cosine_scores[4][4])+"\t"+"{:.4f}".format(cosine_scores[5][5])+"\n"

In [ ]:
f.close()
g = open("/content/drive/My Drive/TFG/Datasets/sts/ratedNewsDevM2M100_418M.csv", "w")
g.write(ratedSentences)
g.close()

we compare the similarity scores of the pairs as s1 s2 should give similar results as ts1 s2 and s1 ts2 and ts1 ts2 if the translator is good, we will also compare it with the original annotated scores of the pair. We add two new columns to the file: the first one compares the similarity against our target similarity obtained with the stsb-mpnet-base-v2 model and the second one against the annotated score(divided by 5 to fit the cosine)

In [ ]:
f = open("/content/drive/My Drive/TFG/Datasets/sts/ratedNewsDevM2M100_418M.csv", "r")
sentences=f.readlines()
simSentences= ""
totalOrigDeviation = 0
totalTargetDeviation = 0
for sentence in sentences:
  prevData = sentence.split("\t")[0:9]
  scores = sentence.split("\t")[9:]
  scores[-1] = scores[-1].replace("\n","")
  similarity = float(scores[0])+float(scores[1])+float(scores[2])+float(scores[3])+float(scores[4])+float(scores[5])
  targetSimilarity = float(scores[0])+1+float(scores[0])+1+float(scores[0])+float(scores[0])
  optimalSimilarity = "{:.4f}".format(float(sentence.split("\t")[4])/5)
  similarity = "{:.4f}".format(similarity/6)
  targetSimilarity = "{:.4f}".format(targetSimilarity/6)
  simTarg = "{:.4f}".format(abs(float(targetSimilarity) - float(similarity)))
  simOrig = "{:.4f}".format(abs(float(optimalSimilarity) - float(similarity)))
  simSentences += prevData[0]+"\t"+prevData[1]+"\t"+prevData[2]+"\t"+prevData[3]+"\t"+prevData[4]+"\t"+prevData[5]+"\t"+prevData[6]+"\t"+prevData[7]+"\t"+prevData[8]+"\t"+"\t"+scores[0]+"\t"+scores[1]+"\t"+scores[2]+"\t"+scores[3]+"\t"+scores[4]+"\t"+scores[5]+"\t"+simTarg+"\t"+simOrig+"\n"
  totalOrigDeviation += float(simOrig)
  totalTargetDeviation += float(simTarg)

In [ ]:
f.close()
g = open("/content/drive/My Drive/TFG/Datasets/sts/ratedSimNewsDevM2M100_418M.csv", "w")
g.write(simSentences)
h = open("/content/drive/My Drive/TFG/Datasets/sts/NewsDevRatingModels.csv", "a")
h.write("M2M100_418M"+"\t"+"{:.4f}".format(totalTargetDeviation/len(sentences))+"\t"+"{:.4f}".format(totalOrigDeviation/len(sentences))+"\n")
g.close()
h.close()

rating the M2M100 1.2B translation

In [ ]:
f = open("/content/drive/My Drive/TFG/Datasets/sts/translatedNewsDevM2M100_1_2B.csv", "r")
sentences=f.readlines()
ratedSentences= ""
for sentence in sentences:
  sentencesSts1 = []
  sentencesSts2 = []
  sentence = sentence.split("\t")
  sentence[-1] = sentence[-1].replace("\n","")
  metaData = sentence[0:5]
  sent = sentence[5:]
  sentencesSts1.extend([sent[0],sent[0],sent[0],sent[1],sent[1],sent[2]])
  sentencesSts2.extend([sent[1],sent[2],sent[3],sent[2],sent[3],sent[3]])
  embeddings1 = model.encode(sentencesSts1, convert_to_tensor=True)
  embeddings2 = model.encode(sentencesSts2, convert_to_tensor=True)
  cosine_scores = util.cos_sim(embeddings1, embeddings2)
  ratedSentences += metaData[0]+"\t"+metaData[1]+"\t"+metaData[2]+"\t"+metaData[3]+"\t"+metaData[4]+"\t"+sent[0]+"\t"+sent[1]+"\t"+sent[2]+"\t"+sent[3]+"\t"+"{:.4f}".format(cosine_scores[0][0])+"\t"+"{:.4f}".format(cosine_scores[1][1])+"\t"+"{:.4f}".format(cosine_scores[2][2])+"\t"+"{:.4f}".format(cosine_scores[3][3])+"\t"+"{:.4f}".format(cosine_scores[4][4])+"\t"+"{:.4f}".format(cosine_scores[5][5])+"\n"

In [ ]:
f.close()
g = open("/content/drive/My Drive/TFG/Datasets/sts/ratedNewsDevM2M100_1_2B.csv", "w")
g.write(ratedSentences)
g.close()

In [ ]:
f = open("/content/drive/My Drive/TFG/Datasets/sts/ratedNewsDevM2M100_1_2B.csv", "r")
sentences=f.readlines()
simSentences= ""
totalOrigDeviation = 0
totalTargetDeviation = 0
for sentence in sentences:
  prevData = sentence.split("\t")[0:9]
  scores = sentence.split("\t")[9:]
  scores[-1] = scores[-1].replace("\n","")
  similarity = float(scores[0])+float(scores[1])+float(scores[2])+float(scores[3])+float(scores[4])+float(scores[5])
  targetSimilarity = float(scores[0])+1+float(scores[0])+1+float(scores[0])+float(scores[0])
  optimalSimilarity = "{:.4f}".format(float(sentence.split("\t")[4])/5)
  similarity = "{:.4f}".format(similarity/6)
  targetSimilarity = "{:.4f}".format(targetSimilarity/6)
  simTarg = "{:.4f}".format(abs(float(targetSimilarity) - float(similarity)))
  simOrig = "{:.4f}".format(abs(float(optimalSimilarity) - float(similarity)))
  simSentences += prevData[0]+"\t"+prevData[1]+"\t"+prevData[2]+"\t"+prevData[3]+"\t"+prevData[4]+"\t"+prevData[5]+"\t"+prevData[6]+"\t"+prevData[7]+"\t"+prevData[8]+"\t"+"\t"+scores[0]+"\t"+scores[1]+"\t"+scores[2]+"\t"+scores[3]+"\t"+scores[4]+"\t"+scores[5]+"\t"+simTarg+"\t"+simOrig+"\n"
  totalOrigDeviation += float(simOrig)
  totalTargetDeviation += float(simTarg)

In [ ]:
f.close()
g = open("/content/drive/My Drive/TFG/Datasets/sts/ratedSimNewsDevM2M100_1_2B.csv", "w")
g.write(simSentences)
h = open("/content/drive/My Drive/TFG/Datasets/sts/NewsDevRatingModels.csv", "a")
h.write("M2M100_1_2B"+"\t"+"{:.4f}".format(totalTargetDeviation/len(sentences))+"\t"+"{:.4f}".format(totalOrigDeviation/len(sentences))+"\n")
h.close()
g.close()

rating the NLLB-200-1.3B

In [ ]:
f = open("/content/drive/My Drive/TFG/Datasets/sts/translatedNewsDevNLLB-200-1_3B.csv", "r")
sentences=f.readlines()
ratedSentences= ""
for sentence in sentences:
  sentencesSts1 = []
  sentencesSts2 = []
  sentence = sentence.split("\t")
  sentence[-1] = sentence[-1].replace("\n","")
  metaData = sentence[0:5]
  sent = sentence[5:]
  sentencesSts1.extend([sent[0],sent[0],sent[0],sent[1],sent[1],sent[2]])
  sentencesSts2.extend([sent[1],sent[2],sent[3],sent[2],sent[3],sent[3]])
  embeddings1 = model.encode(sentencesSts1, convert_to_tensor=True)
  embeddings2 = model.encode(sentencesSts2, convert_to_tensor=True)
  cosine_scores = util.cos_sim(embeddings1, embeddings2)
  ratedSentences += metaData[0]+"\t"+metaData[1]+"\t"+metaData[2]+"\t"+metaData[3]+"\t"+metaData[4]+"\t"+sent[0]+"\t"+sent[1]+"\t"+sent[2]+"\t"+sent[3]+"\t"+"{:.4f}".format(cosine_scores[0][0])+"\t"+"{:.4f}".format(cosine_scores[1][1])+"\t"+"{:.4f}".format(cosine_scores[2][2])+"\t"+"{:.4f}".format(cosine_scores[3][3])+"\t"+"{:.4f}".format(cosine_scores[4][4])+"\t"+"{:.4f}".format(cosine_scores[5][5])+"\n"

In [ ]:
f.close()
g = open("/content/drive/My Drive/TFG/Datasets/sts/ratedNewsDevNLLB-200-1_3B.csv", "w")
g.write(ratedSentences)
g.close()

In [ ]:
f = open("/content/drive/My Drive/TFG/Datasets/sts/ratedNewsDevNLLB-200-1_3B.csv", "r")
sentences=f.readlines()
simSentences= ""
totalOrigDeviation = 0
totalTargetDeviation = 0
for sentence in sentences:
  prevData = sentence.split("\t")[0:9]
  scores = sentence.split("\t")[9:]
  scores[-1] = scores[-1].replace("\n","")
  similarity = float(scores[0])+float(scores[1])+float(scores[2])+float(scores[3])+float(scores[4])+float(scores[5])
  targetSimilarity = float(scores[0])+1+float(scores[0])+1+float(scores[0])+float(scores[0])
  optimalSimilarity = "{:.4f}".format(float(sentence.split("\t")[4])/5)
  similarity = "{:.4f}".format(similarity/6)
  targetSimilarity = "{:.4f}".format(targetSimilarity/6)
  simTarg = "{:.4f}".format(abs(float(targetSimilarity) - float(similarity)))
  simOrig = "{:.4f}".format(abs(float(optimalSimilarity) - float(similarity)))
  simSentences += prevData[0]+"\t"+prevData[1]+"\t"+prevData[2]+"\t"+prevData[3]+"\t"+prevData[4]+"\t"+prevData[5]+"\t"+prevData[6]+"\t"+prevData[7]+"\t"+prevData[8]+"\t"+"\t"+scores[0]+"\t"+scores[1]+"\t"+scores[2]+"\t"+scores[3]+"\t"+scores[4]+"\t"+scores[5]+"\t"+simTarg+"\t"+simOrig+"\n"
  totalOrigDeviation += float(simOrig)
  totalTargetDeviation += float(simTarg)

In [ ]:
f.close()
g = open("/content/drive/My Drive/TFG/Datasets/sts/ratedSimNewsDevNLLB-200-1_3B.csv", "w")
g.write(simSentences)
h = open("/content/drive/My Drive/TFG/Datasets/sts/NewsDevRatingModels.csv", "a")
h.write("NLLB-200-1_3B"+"\t"+"{:.4f}".format(totalTargetDeviation/len(sentences))+"\t"+"{:.4f}".format(totalOrigDeviation/len(sentences))+"\n")
h.close()
g.close()

rating the NLLB-200-distilled-600M

In [ ]:
f = open("/content/drive/My Drive/TFG/Datasets/sts/translatedNewsDevNLLB-200-distilled-600M.csv", "r")
sentences=f.readlines()
ratedSentences= ""
for sentence in sentences:
  sentencesSts1 = []
  sentencesSts2 = []
  sentence = sentence.split("\t")
  sentence[-1] = sentence[-1].replace("\n","")
  metaData = sentence[0:5]
  sent = sentence[5:]
  sentencesSts1.extend([sent[0],sent[0],sent[0],sent[1],sent[1],sent[2]])
  sentencesSts2.extend([sent[1],sent[2],sent[3],sent[2],sent[3],sent[3]])
  embeddings1 = model.encode(sentencesSts1, convert_to_tensor=True)
  embeddings2 = model.encode(sentencesSts2, convert_to_tensor=True)
  cosine_scores = util.cos_sim(embeddings1, embeddings2)
  ratedSentences += metaData[0]+"\t"+metaData[1]+"\t"+metaData[2]+"\t"+metaData[3]+"\t"+metaData[4]+"\t"+sent[0]+"\t"+sent[1]+"\t"+sent[2]+"\t"+sent[3]+"\t"+"{:.4f}".format(cosine_scores[0][0])+"\t"+"{:.4f}".format(cosine_scores[1][1])+"\t"+"{:.4f}".format(cosine_scores[2][2])+"\t"+"{:.4f}".format(cosine_scores[3][3])+"\t"+"{:.4f}".format(cosine_scores[4][4])+"\t"+"{:.4f}".format(cosine_scores[5][5])+"\n"

In [ ]:
f.close()
g = open("/content/drive/My Drive/TFG/Datasets/sts/ratedNewsDevNLLB-200-distilled-600M.csv", "w")
g.write(ratedSentences)
g.close()

In [ ]:
f = open("/content/drive/My Drive/TFG/Datasets/sts/ratedNewsDevNLLB-200-distilled-600M.csv", "r")
sentences=f.readlines()
simSentences= ""
totalOrigDeviation = 0
totalTargetDeviation = 0
for sentence in sentences:
  prevData = sentence.split("\t")[0:9]
  scores = sentence.split("\t")[9:]
  scores[-1] = scores[-1].replace("\n","")
  similarity = float(scores[0])+float(scores[1])+float(scores[2])+float(scores[3])+float(scores[4])+float(scores[5])
  targetSimilarity = float(scores[0])+1+float(scores[0])+1+float(scores[0])+float(scores[0])
  optimalSimilarity = "{:.4f}".format(float(sentence.split("\t")[4])/5)
  similarity = "{:.4f}".format(similarity/6)
  targetSimilarity = "{:.4f}".format(targetSimilarity/6)
  simTarg = "{:.4f}".format(abs(float(targetSimilarity) - float(similarity)))
  simOrig = "{:.4f}".format(abs(float(optimalSimilarity) - float(similarity)))
  simSentences += prevData[0]+"\t"+prevData[1]+"\t"+prevData[2]+"\t"+prevData[3]+"\t"+prevData[4]+"\t"+prevData[5]+"\t"+prevData[6]+"\t"+prevData[7]+"\t"+prevData[8]+"\t"+"\t"+scores[0]+"\t"+scores[1]+"\t"+scores[2]+"\t"+scores[3]+"\t"+scores[4]+"\t"+scores[5]+"\t"+simTarg+"\t"+simOrig+"\n"
  totalOrigDeviation += float(simOrig)
  totalTargetDeviation += float(simTarg)

In [ ]:
f.close()
g = open("/content/drive/My Drive/TFG/Datasets/sts/ratedSimNewsDevNLLB-200-distilled-600M.csv", "w")
g.write(simSentences)
h = open("/content/drive/My Drive/TFG/Datasets/sts/NewsDevRatingModels.csv", "a")
h.write("NLLB-200-distilled-600M"+"\t"+"{:.4f}".format(totalTargetDeviation/len(sentences))+"\t"+"{:.4f}".format(totalOrigDeviation/len(sentences))+"\n")
h.close()
g.close()

rating the opus-mt-en-es

In [ ]:
f = open("/content/drive/My Drive/TFG/Datasets/sts/translatedNewsDevOpus-mt-en-es.csv", "r")
sentences=f.readlines()
ratedSentences= ""
for sentence in sentences:
  sentencesSts1 = []
  sentencesSts2 = []
  sentence = sentence.split("\t")
  sentence[-1] = sentence[-1].replace("\n","")
  metaData = sentence[0:5]
  sent = sentence[5:]
  sentencesSts1.extend([sent[0],sent[0],sent[0],sent[1],sent[1],sent[2]])
  sentencesSts2.extend([sent[1],sent[2],sent[3],sent[2],sent[3],sent[3]])
  embeddings1 = model.encode(sentencesSts1, convert_to_tensor=True)
  embeddings2 = model.encode(sentencesSts2, convert_to_tensor=True)
  cosine_scores = util.cos_sim(embeddings1, embeddings2)
  ratedSentences += metaData[0]+"\t"+metaData[1]+"\t"+metaData[2]+"\t"+metaData[3]+"\t"+metaData[4]+"\t"+sent[0]+"\t"+sent[1]+"\t"+sent[2]+"\t"+sent[3]+"\t"+"{:.4f}".format(cosine_scores[0][0])+"\t"+"{:.4f}".format(cosine_scores[1][1])+"\t"+"{:.4f}".format(cosine_scores[2][2])+"\t"+"{:.4f}".format(cosine_scores[3][3])+"\t"+"{:.4f}".format(cosine_scores[4][4])+"\t"+"{:.4f}".format(cosine_scores[5][5])+"\n"

In [ ]:
f.close()
g = open("/content/drive/My Drive/TFG/Datasets/sts/ratedNewsDevOpus-mt-en-es.csv", "w")
g.write(ratedSentences)
g.close()

In [ ]:
f = open("/content/drive/My Drive/TFG/Datasets/sts/ratedNewsDevOpus-mt-en-es.csv", "r")
sentences=f.readlines()
simSentences= ""
totalOrigDeviation = 0
totalTargetDeviation = 0
for sentence in sentences:
  prevData = sentence.split("\t")[0:9]
  scores = sentence.split("\t")[9:]
  scores[-1] = scores[-1].replace("\n","")
  similarity = float(scores[0])+float(scores[1])+float(scores[2])+float(scores[3])+float(scores[4])+float(scores[5])
  targetSimilarity = float(scores[0])+1+float(scores[0])+1+float(scores[0])+float(scores[0])
  optimalSimilarity = "{:.4f}".format(float(sentence.split("\t")[4])/5)
  similarity = "{:.4f}".format(similarity/6)
  targetSimilarity = "{:.4f}".format(targetSimilarity/6)
  simTarg = "{:.4f}".format(abs(float(targetSimilarity) - float(similarity)))
  simOrig = "{:.4f}".format(abs(float(optimalSimilarity) - float(similarity)))
  simSentences += prevData[0]+"\t"+prevData[1]+"\t"+prevData[2]+"\t"+prevData[3]+"\t"+prevData[4]+"\t"+prevData[5]+"\t"+prevData[6]+"\t"+prevData[7]+"\t"+prevData[8]+"\t"+"\t"+scores[0]+"\t"+scores[1]+"\t"+scores[2]+"\t"+scores[3]+"\t"+scores[4]+"\t"+scores[5]+"\t"+simTarg+"\t"+simOrig+"\n"
  totalOrigDeviation += float(simOrig)
  totalTargetDeviation += float(simTarg)

In [ ]:
f.close()
g = open("/content/drive/My Drive/TFG/Datasets/sts/ratedSimNewsDevOpus-mt-en-es.csv", "w")
g.write(simSentences)
h = open("/content/drive/My Drive/TFG/Datasets/sts/NewsDevRatingModels.csv", "a")
h.write("Opus-mt-en-es"+"\t"+"{:.4f}".format(totalTargetDeviation/len(sentences))+"\t"+"{:.4f}".format(totalOrigDeviation/len(sentences))+"\n")
h.close()
g.close()

rating the opus-mt-en-ROMANCE

In [ ]:
f = open("/content/drive/My Drive/TFG/Datasets/sts/translatedNewsDevOpus-mt-en-ROMANCE.csv", "r")
sentences=f.readlines()
ratedSentences= ""
for sentence in sentences:
  sentencesSts1 = []
  sentencesSts2 = []
  sentence = sentence.split("\t")
  sentence[-1] = sentence[-1].replace("\n","")
  metaData = sentence[0:5]
  sent = sentence[5:]
  sentencesSts1.extend([sent[0],sent[0],sent[0],sent[1],sent[1],sent[2]])
  sentencesSts2.extend([sent[1],sent[2],sent[3],sent[2],sent[3],sent[3]])
  embeddings1 = model.encode(sentencesSts1, convert_to_tensor=True)
  embeddings2 = model.encode(sentencesSts2, convert_to_tensor=True)
  cosine_scores = util.cos_sim(embeddings1, embeddings2)
  ratedSentences += metaData[0]+"\t"+metaData[1]+"\t"+metaData[2]+"\t"+metaData[3]+"\t"+metaData[4]+"\t"+sent[0]+"\t"+sent[1]+"\t"+sent[2]+"\t"+sent[3]+"\t"+"{:.4f}".format(cosine_scores[0][0])+"\t"+"{:.4f}".format(cosine_scores[1][1])+"\t"+"{:.4f}".format(cosine_scores[2][2])+"\t"+"{:.4f}".format(cosine_scores[3][3])+"\t"+"{:.4f}".format(cosine_scores[4][4])+"\t"+"{:.4f}".format(cosine_scores[5][5])+"\n"

In [ ]:
f.close()
g = open("/content/drive/My Drive/TFG/Datasets/sts/ratedNewsDevOpus-mt-en-ROMANCE.csv", "w")
g.write(ratedSentences)
g.close()

In [ ]:
f = open("/content/drive/My Drive/TFG/Datasets/sts/ratedNewsDevOpus-mt-en-ROMANCE.csv", "r")
sentences=f.readlines()
simSentences= ""
totalOrigDeviation = 0
totalTargetDeviation = 0
for sentence in sentences:
  prevData = sentence.split("\t")[0:9]
  scores = sentence.split("\t")[9:]
  scores[-1] = scores[-1].replace("\n","")
  similarity = float(scores[0])+float(scores[1])+float(scores[2])+float(scores[3])+float(scores[4])+float(scores[5])
  targetSimilarity = float(scores[0])+1+float(scores[0])+1+float(scores[0])+float(scores[0])
  optimalSimilarity = "{:.4f}".format(float(sentence.split("\t")[4])/5)
  similarity = "{:.4f}".format(similarity/6)
  targetSimilarity = "{:.4f}".format(targetSimilarity/6)
  simTarg = "{:.4f}".format(abs(float(targetSimilarity) - float(similarity)))
  simOrig = "{:.4f}".format(abs(float(optimalSimilarity) - float(similarity)))
  simSentences += prevData[0]+"\t"+prevData[1]+"\t"+prevData[2]+"\t"+prevData[3]+"\t"+prevData[4]+"\t"+prevData[5]+"\t"+prevData[6]+"\t"+prevData[7]+"\t"+prevData[8]+"\t"+"\t"+scores[0]+"\t"+scores[1]+"\t"+scores[2]+"\t"+scores[3]+"\t"+scores[4]+"\t"+scores[5]+"\t"+simTarg+"\t"+simOrig+"\n"
  totalOrigDeviation += float(simOrig)
  totalTargetDeviation += float(simTarg)

In [ ]:
f.close()
g = open("/content/drive/My Drive/TFG/Datasets/sts/ratedSimNewsDevOpus-mt-en-ROMANCE.csv", "w")
g.write(simSentences)
h = open("/content/drive/My Drive/TFG/Datasets/sts/NewsDevRatingModels.csv", "a")
h.write("Opus-mt-en-ROMANCE"+"\t"+"{:.4f}".format(totalTargetDeviation/len(sentences))+"\t"+"{:.4f}".format(totalOrigDeviation/len(sentences))+"\n")
h.close()
g.close()